In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: DataSonogramMBS – Data Sonogram Model Based Sonification

This notebook introduces and demonstrates usage of `DataSonogramMBS`, the Data Sonogram Model Based Sonification sonecule.
* The sonecule creates a 2D plot showing the first two dimensions of the data frame
* A `on_click` interaction is bound to the plot allowing users to excite the sonification model.
* On a Mouse click at any position in the plot the nearest neighbor in the 2D scatter is identified
* Then a shockwave is triggered to emanate from that location, however, it spreads as $d$-dimensional ball within the $d$-dimensional data space
* as the shock wave front reaches a given data point, that point – imagined as a mass-spring system – begins to oscillate due to energy transfer
* in result we hear a spherical scan of the data set starting from the location of excitation
* The stereo position is determined from the relative location of the data point in the displayed projection relevative to the excitation center
* model parameters are 
  * the shock wave velocity, 
  * the global level
  * the ring time, i.e. the 60 dB decay times in seconds for the mass-spring systems of each data point.
  * and a mode: which is currently unused, but will later allow to select a task-specific model variation. For instance `"kNN-entropy"` shall use the entropy of class labels among the $k$ nearest neighbors of each data point as spring stiffness, resulting in higher-pitched tones for data points that do not find themselves in a homogeneous area. This allows for instance to detect/inspect class borders and class overlaps.

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx
ctx.enable_realtime();

Load data sets used for the demo

In [ ]:
%run ../data/prepare-data.ipynb

In [ ]:
sns.pairplot(data=penguins_df, hue="species", height=1.2);

In [ ]:
df = dataframes['penguins']
df.columns

## Usage Demo for the Data Sonogram Model-Based Sonification Sonecule

In [ ]:
from sonecules.triggerson import DataSonogramMBS

The following code cell shows everything needed 
- to create the sonecule with data, 
- to reset the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

After executing the following line, click at any location in the plot with the left Mouse button (tap the track pad).
The sonification starts with a noisy transient to indicate the exact moment in time when the shock wave started.
This is followed by the sound of all 

In [ ]:
dsg = DataSonogramMBS(penguins_df, x="flipper_length_mm", y="bill_length_mm", label="species", rtime=1).start()

In [ ]:
# here a GUI to control the parameters - continue clicking in the plot
def dsg_gui(rtime=0.5, max_duration=3, level=-6, trigger_sound=True):
    dsg.rtime, dsg.max_duration, dsg.level, dsg.play_trigger_sound = rtime, max_duration, level, trigger_sound

from ipywidgets import interactive
interactive(dsg_gui, rtime=(0.05,5, 0.01), max_duration=(0.2, 10, 0.1), level=(-40, 10, 1))


In [ ]:
dsg._latency = 0.1

In [ ]:
dsg.play_trigger_sound 

In [ ]:
# You can disable the trigger sound and continue to click in the above plot without noise sample
dsg.play_trigger_sound = False

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an Asig dasig

In [ ]:
from sonecules.triggerson import DataSonogramMBS
import numpy as np
import pandas as pd

# load your multi-channel data into data frame: here 400 points in 4 clusters
data = np.random.randn(400, 4) * np.tile(np.linspace(1, 0.2, 400), (4, 1)).T
data[:100,:]    += np.tile([2,2,0,0],(100,1))
data[100:200,:] += np.tile([1,4.5,0,0],(100,1))
data[200:300,:] += np.tile([3,4.5,0,0],(100,1))
data[:, 2] = 0; data[100:200,2] = 1; data[200:,2] = 2; data[300:,2] = 3
df = pd.DataFrame(data, columns=[0,1,2,3])

# enable realtims sonification
ctx.enable_realtime()

# create the model and GUI -> then click in the plot
dsg = DataSonogramMBS(df, x=0, y=1, label=2)

In [ ]:
# now control parameters such as ring time or maximal duration 
dsg.rtime = 6.5 
dsg.max_duration = 3
dsg.level=-18
dsg.play_trigger_sound = False
# keep on clicking in the plot with new settings.

In [ ]:
# here a GUI to control the parameters
def dsg_gui(rtime=0.5, max_duration=3, level=-6, trigger_sound=True):
    dsg.rtime, dsg.max_duration, dsg.level, dsg.play_trigger_sound = rtime, max_duration, level, trigger_sound

from ipywidgets import interactive
interactive(dsg_gui, rtime=(0.05,5, 0.01), max_duration=(0.2, 10, 0.1), level=(-40, 10, 1))


In [ ]:
ctx.close()